In [1]:
import random
random.seed(10)

In [2]:
import re
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import *

In [3]:
data = open("data.txt", "r").read()

In [4]:
type(data)

str

In [5]:
print(data)

'Olá, como vai? Eu sou a Camila.\n'
'Olá, Camila, meu nome é Fernando. Muito prazer.\n'
'Prazer em conhecer você também. Como você está hoje?\n'
'Ótimo. Meu time de futebol venceu a competição.\n'
'Uau, Parabéns Fernando!\n'
'Obrigado Camila.\n'
'Vamos comer uma pizza mais tarde para celebrar?\n'
'Claro. Você recomenda algum restaurante Camila?\n'
'Sim, abriu um restaurante novo e dizem que a pizza de banana é fenomenal.\n'
'Ok. Nos encontramos no restaurante às sete da noite, pode ser?\n'
'Pode sim. Nos vemos mais tarde então.'


## vocab construction

In [45]:
# re.sub(...) function replaces all occurrences of the characters within the character class with an empty string
sentences = re.sub("[.,!?\\-']", '', data.lower().replace("\n", " ")).split('\\n')
sentences = [s.strip() for s in sentences]

print(sentences)

['olá como vai eu sou a camila', 'olá camila meu nome é fernando muito prazer', 'prazer em conhecer você também como você está hoje', 'ótimo meu time de futebol venceu a competição', 'uau parabéns fernando', 'obrigado camila', 'vamos comer uma pizza mais tarde para celebrar', 'claro você recomenda algum restaurante camila', 'sim abriu um restaurante novo e dizem que a pizza de banana é fenomenal', 'ok nos encontramos no restaurante às sete da noite pode ser', 'pode sim nos vemos mais tarde então']


In [28]:
words_lst = list( " ".join(sentences).split() )

print(words_lst)

['olá', 'como', 'vai', 'eu', 'sou', 'a', 'camila', 'olá', 'camila', 'meu', 'nome', 'é', 'fernando', 'muito', 'prazer', 'prazer', 'em', 'conhecer', 'você', 'também', 'como', 'você', 'está', 'hoje', 'ótimo', 'meu', 'time', 'de', 'futebol', 'venceu', 'a', 'competição', 'uau', 'parabéns', 'fernando', 'obrigado', 'camila', 'vamos', 'comer', 'uma', 'pizza', 'mais', 'tarde', 'para', 'celebrar', 'claro', 'você', 'recomenda', 'algum', 'restaurante', 'camila', 'sim', 'abriu', 'um', 'restaurante', 'novo', 'e', 'dizem', 'que', 'a', 'pizza', 'de', 'banana', 'é', 'fenomenal', 'ok', 'nos', 'encontramos', 'no', 'restaurante', 'às', 'sete', 'da', 'noite', 'pode', 'ser', 'pode', 'sim', 'nos', 'vemos', 'mais', 'tarde', 'então']


In [29]:
# BERT special tokens
words_dict = {
    "[PAD]": 0, 
    "[CLS]": 1, 
    "[SEP]": 2, 
    "[MASK]": 3
    }

In [30]:
for i, w in enumerate(words_lst):
    words_dict[w] = i + 4

In [31]:
print(words_dict)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, 'olá': 11, 'como': 24, 'vai': 6, 'eu': 7, 'sou': 8, 'a': 63, 'camila': 54, 'meu': 29, 'nome': 14, 'é': 67, 'fernando': 38, 'muito': 17, 'prazer': 19, 'em': 20, 'conhecer': 21, 'você': 50, 'também': 23, 'está': 26, 'hoje': 27, 'ótimo': 28, 'time': 30, 'de': 65, 'futebol': 32, 'venceu': 33, 'competição': 35, 'uau': 36, 'parabéns': 37, 'obrigado': 39, 'vamos': 41, 'comer': 42, 'uma': 43, 'pizza': 64, 'mais': 84, 'tarde': 85, 'para': 47, 'celebrar': 48, 'claro': 49, 'recomenda': 51, 'algum': 52, 'restaurante': 73, 'sim': 81, 'abriu': 56, 'um': 57, 'novo': 59, 'e': 60, 'dizem': 61, 'que': 62, 'banana': 66, 'fenomenal': 68, 'ok': 69, 'nos': 82, 'encontramos': 71, 'no': 72, 'às': 74, 'sete': 75, 'da': 76, 'noite': 77, 'pode': 80, 'ser': 79, 'vemos': 83, 'então': 86}


In [32]:
nums_dict = { i: k for k, i in words_dict.items()}

print(nums_dict)

{0: '[PAD]', 1: '[CLS]', 2: '[SEP]', 3: '[MASK]', 11: 'olá', 24: 'como', 6: 'vai', 7: 'eu', 8: 'sou', 63: 'a', 54: 'camila', 29: 'meu', 14: 'nome', 67: 'é', 38: 'fernando', 17: 'muito', 19: 'prazer', 20: 'em', 21: 'conhecer', 50: 'você', 23: 'também', 26: 'está', 27: 'hoje', 28: 'ótimo', 30: 'time', 65: 'de', 32: 'futebol', 33: 'venceu', 35: 'competição', 36: 'uau', 37: 'parabéns', 39: 'obrigado', 41: 'vamos', 42: 'comer', 43: 'uma', 64: 'pizza', 84: 'mais', 85: 'tarde', 47: 'para', 48: 'celebrar', 49: 'claro', 51: 'recomenda', 52: 'algum', 73: 'restaurante', 81: 'sim', 56: 'abriu', 57: 'um', 59: 'novo', 60: 'e', 61: 'dizem', 62: 'que', 66: 'banana', 68: 'fenomenal', 69: 'ok', 82: 'nos', 71: 'encontramos', 72: 'no', 74: 'às', 75: 'sete', 76: 'da', 77: 'noite', 80: 'pode', 79: 'ser', 83: 'vemos', 86: 'então'}


In [34]:
VOCAB_SIZE = len(words_dict)
VOCAB_SIZE

65

In [47]:
sentences[0].split()

['olá', 'como', 'vai', 'eu', 'sou', 'a', 'camila']

In [49]:
tokens_lst = []

for sentence in sentences:
    tokens = [words_dict[s] for s in sentence.split()]
    tokens_lst.append(tokens)

In [50]:
tokens_lst

[[11, 24, 6, 7, 8, 63, 54],
 [11, 54, 29, 14, 67, 38, 17, 19],
 [19, 20, 21, 50, 23, 24, 50, 26, 27],
 [28, 29, 30, 65, 32, 33, 63, 35],
 [36, 37, 38],
 [39, 54],
 [41, 42, 43, 64, 84, 85, 47, 48],
 [49, 50, 51, 52, 73, 54],
 [81, 56, 57, 73, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68],
 [69, 82, 71, 72, 73, 74, 75, 76, 77, 80, 79],
 [80, 81, 82, 83, 84, 85, 86]]

## hyperparams definition

In [51]:
BATCH_SIZE = 6
N_SEGMENTS = 2
DROPOUT    = 0.2

MAX_LEN     = 100
MAX_PRED    = 7 # max num of tokens that will be predicted 

N_LAYERS    = 6
N_HEADS     = 12

D_MODEL     = 768 # embedding dim
D_FF        = D_MODEL * 4 # feedforward dim
D_K = D_V   = 64

N_EPOCHS    = 10